In [4]:
import cv2
import os
import pandas as pd
from PIL import Image
import numpy as np
import re
import glob

In [5]:
img_dir = r'C:\Users\stbyu\Desktop\University of Michigan\CurrQuarter\EECS 545\FinalProject\train'
search_path = os.path.join(img_dir, "**",  "*.png")
files = glob.glob(search_path, recursive=True)
img_files = [f.replace('\\', '/') for f in files]
print(len(img_files))

#img_files = img_files[28000:]

38496


In [6]:
def pad_image(img_arr, height, width):
    # Calculate the amount of padding needed
    h, w = img_arr.shape[:2]
    pad_h = max(0, height - h)
    pad_w = max(0, width - w)

    # Add padding to the image
    padded_arr = np.pad(img_arr, ((0, pad_h), (0, pad_w)), mode='constant', constant_values=0)
    return(padded_arr)


In [7]:
filename = []
maxh = 310
maxw = 360
image_data = np.zeros((len(img_files), maxh, maxw),dtype=np.uint8)
shape = []

for i,img_path in enumerate(img_files):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # Convert to grayscale
    img_np = np.asarray(img)
    label1 = re.search(r'case[\d]+_day[\d]+', img_path)
    label2 = re.search(r'slice_[\d]+', img_path)
    tmp_list = img_path.split('_')
    shape.append((int(tmp_list[4]), int(tmp_list[3])))
    fn = label1[0]+"_"+label2[0]
    image_data[i] = pad_image(img_np, maxh, maxw)
    filename.append(fn)
    if (i%1000 == 0):
        print(i)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000


In [11]:
tmp_shape = len(img_files) /2

In [12]:
tmp_shape = int(tmp_shape)
img2_5D = np.zeros((tmp_shape, 3, maxh, maxw), dtype = int)
labels = np.zeros((tmp_shape, 3, maxh, maxw), dtype = int)

In [13]:
def rle_decode(mask_rle, shape=(768, 768)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return
    Returns numpy array, 1 - mask, 0 - background
    '''
    if not isinstance(mask_rle, str):
        img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
        return img.reshape(shape)

    s = mask_rle.split()
    s = s[1:]
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)

In [21]:
path = r'C:\Users\stbyu\Desktop\University of Michigan\CurrQuarter\EECS 545\FinalProject\train\train.csv'
path = path.replace('\\', '/')
df = pd.read_csv(path)
pd.options.display.max_colwidth = None
j = 0
for i in range(len(img_files)):
    if (i % 1000 ==0):
        print(i, j)
    # process the img2.5D
    if (i == 0 or i == len(img_files) - 1):
        continue
    pre_sn = re.search(r"(\w+_\w+)_slice", filename[i-1]).group(1)
    curr_sn = re.search(r"(\w+_\w+)_slice", filename[i]).group(1)
    next_sn = re.search(r"(\w+_\w+)_slice", filename[i+1]).group(1)
    if (pre_sn.lower() != curr_sn.lower()):
        continue
    if(next_sn.lower() != curr_sn.lower()):
        continue
        
    img2_5D[j,1,:,:] = image_data[i]
    img2_5D[j,0,:,:] = image_data[i-1]
    img2_5D[j,2,:,:] = image_data[i+1]

  # process the masks aka labels
    fn = filename[i]
    valid = 0
    filtered_df = df[(df['id'] == fn) & (df['class'] == 'large_bowel')]
    if (not filtered_df["segmentation"].isna().all()):
        valid = 1
        labels[j,0,:,:] = pad_image(rle_decode(filtered_df["segmentation"].to_string(), shape[i]), maxh, maxw)
    
    filtered_df = df[(df['id'] == fn) & (df['class'] == 'small_bowel')]
    if (not filtered_df["segmentation"].isna().all()):
        valid = 1
        labels[j,1,:,:] = pad_image(rle_decode(filtered_df["segmentation"].to_string(), shape[i]), maxh, maxw)
    
    filtered_df = df[(df['id'] == fn) & (df['class'] == 'stomach')]
    if (not filtered_df["segmentation"].isna().all()):
        valid = 1
        labels[j,2,:,:] = pad_image(rle_decode(filtered_df["segmentation"].to_string(), shape[i]), maxh, maxw)
    
    if (valid):
        j = j + 1
    if (j >= len(img_files)/2): 
        break

0 0
1000 418
2000 898
3000 1359
4000 1806
5000 2345
6000 2763
7000 3257
8000 3709
9000 4202
10000 4669
11000 5172
12000 5650
13000 6073
14000 6543
15000 6950
16000 7308
17000 7692
18000 8092
19000 8495
20000 9025
21000 9517
22000 9845
23000 10235
24000 10700
25000 11235
26000 11646
27000 12118
28000 12548
29000 12991
30000 13365
31000 13775
32000 14199
33000 14486
34000 14797
35000 15170
36000 15523
37000 15874
38000 16361


In [22]:
img2_5D = img2_5D[:j, :,:,:]
labels = labels[:j, :, :, :]

In [23]:
np.save("New_input_data.npy", img2_5D)
np.save("New_label_data.npy", labels)

In [24]:
print(j / len(img_files))

0.43095386533665836


In [29]:
from sklearn.model_selection import train_test_split

def split_dataset(data, labels, train_ratio=0.7, val_ratio=0.2, test_ratio=0.1):
#     if train_ratio + val_ratio + test_ratio != 1.0:
#         raise ValueError("The sum of ratios must be equal to 1")

    # Split data into training and temp sets (validation + testing)
    train_data, temp_data, train_labels, temp_labels = train_test_split(
        data, labels, train_size=train_ratio, random_state=545
    )

    # Calculate the ratio of validation set relative to the temp set (validation + testing)
    relative_val_ratio = val_ratio / (val_ratio + test_ratio)

    # Split the temp set into validation and testing sets
    val_data, test_data, val_labels, test_labels = train_test_split(
        temp_data, temp_labels, train_size=relative_val_ratio, random_state=545
    )

    return (train_data, train_labels), (val_data, val_labels), (test_data, test_labels)

In [30]:
# indices = np.random.permutation(j)
# training_idx, val_idx, test_idx = indices[:int(0.7*j)], indices[int(0.7*j):int(0.9*j)], indices[int(0.9*j):] 

In [31]:
# data = [...]  # Your dataset features
# labels = [...]  # Your dataset labels
(train_data, train_labels), (val_data, val_labels), (test_data, test_labels) = split_dataset(img2_5D, labels)

In [32]:
print(img2_5D.shape, labels.shape)
# np.save("train/undivided_data_2.npy", img2_5D)
# np.save("train/undivided_label_2.npy", labels)

(16590, 3, 310, 360) (16590, 3, 310, 360)


In [ ]:
# training_data, val_data, test_data = img2_5D[training_idx, :,:,:], img2_5D[val_idx, :,:,:], img2_5D[test_idx, :,:,:] 
# training_label, val_label, test_label = labels[training_idx, :,:,:], labels[val_idx, :,:,:], labels[test_idx, :,:,:] 

In [33]:
print(train_data.shape, val_data.shape, test_data.shape)
print(train_labels.shape, val_labels.shape, test_labels.shape)

(11613, 3, 310, 360) (3318, 3, 310, 360) (1659, 3, 310, 360)
(11613, 3, 310, 360) (3318, 3, 310, 360) (1659, 3, 310, 360)


In [34]:
np.save("train_input.npy", train_data)
np.save("val_input.npy", val_data)
np.save("test_input.npy", test_data)

In [35]:
np.save("train_labels.npy", train_labels)
np.save("val_labels.npy", val_labels)
np.save("test_labels.npy", test_labels)